# Preprocess

In [1]:
import pandas as pd
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tseh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tseh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tseh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [68]:
CVs = pd.read_csv("C:\\Users\\Tseh\\Downloads\\resumes.csv", delimiter = ';', nrows=20000)
vacancies = pd.read_csv("C:\\Users\\Tseh\\Downloads\\vacancies.csv", nrows=20000)

In [69]:
CVs = CVs.drop(columns = ['Пол, возраст', 'ЗП', 'Обновление резюме', 'Авто'])
CVs['resume'] = CVs.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
CVs = CVs[['resume']]
CVs.insert(0, 'id', range(1, len(CVs) + 1))
CVs.head()

,id,resume
0,1,Системный администратор Советск (Калининградск...
1,2,"Технический писатель Королев , не готов к пере..."
2,3,"Оператор Тверь , не готова к переезду , не гот..."
3,4,Веб-разработчик (HTML / CSS / JS / PHP / базы ...
4,5,"Региональный менеджер по продажам Москва , не ..."


In [70]:
vacancies = vacancies.drop(columns = ['Ids', 'Employer', 'Salary', 'From', 'To', 'Published at', 'Professional roles', 'Specializations', 'Profarea names'])
vacancies['vacancies'] = vacancies.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
vacancies = vacancies[['vacancies']]
vacancies.insert(0, 'id', range(1, len(vacancies) + 1))
vacancies.head()

,id,vacancies
0,1,Golang Developer (Кипр) От 3 до 6 лет Полный д...
1,2,Е-mail маркетолог От 1 года до 3 лет Полный де...
2,3,Оператор call-центра (удаленно) От 1 года до 3...
3,4,Ведущий SMM специалист От 1 года до 3 лет Полн...
4,5,UX/UI Designer От 1 года до 3 лет Полный день ...


In [71]:
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(str(text).lower())   

    # stop words  
    stop_words = set(stopwords.words('russian'))
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

In [85]:
CVs['Processed_Resume'] = CVs['resume'].apply(preprocess_text)
vacancies['Processed_Description'] = vacancies['vacancies'].apply(preprocess_text)

# TFid Vectorization

In [62]:
# Vectorization
tfidf = TfidfVectorizer()
tfidf_matrix_cv = tfidf.fit_transform(CVs['Processed_Resume'])
tfidf_matrix_vacancies = tfidf.transform(vacancies['Processed_Description'])

In [63]:
# cosine similarity 
similarity_matrix = cosine_similarity(tfidf_matrix_cv, tfidf_matrix_vacancies)

matched_indices = similarity_matrix.argmax(axis=1)
CVs['Matched_Vacancy_ID'] = matched_indices

In [64]:
CVs['Matched_Vacancy_Description'] = vacancies.loc[matched_indices, 'vacancies'].values

In [65]:
# new matched dataset   
matched_data = pd.merge(CVs, vacancies, left_on='Matched_Vacancy_ID', right_index=True, suffixes=('_Resume', '_Vacancy'))

In [66]:
matched_data_final = matched_data[['Processed_Resume', 'Processed_Description']]

In [67]:
matched_data_final.head(50)

,Processed_Resume,Processed_Description
0,системный администратор советск калининградска...,системный администратор года лет полный день п...
439,системный администратор омск готов переезду го...,системный администратор года лет полный день п...
1800,системный администратор пенза готов переезду г...,системный администратор года лет полный день п...
2193,системный администратор астана готов переезду ...,системный администратор года лет полный день п...
7135,системный администратор дзержинск нижегородска...,системный администратор года лет полный день п...
8937,системный администратор помощник системного ад...,системный администратор года лет полный день п...
9012,системный администратор уфа готов переезду гот...,системный администратор года лет полный день п...
12827,системный администратор уфа готов переезду гот...,системный администратор года лет полный день п...
13362,системный администратор готов переезду готов к...,системный администратор года лет полный день п...
18742,системный администратор волжский готов переезд...,системный администратор года лет полный день п...


# Doc2Vec

In [76]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [77]:
tagged_data_cv = [TaggedDocument(words=doc, tags=[i]) for i, doc in enumerate(CVs['Processed_Resume'])]
tagged_data_vacancies = [TaggedDocument(words=doc, tags=[i]) for i, doc in enumerate(vacancies['Processed_Description'])]

In [78]:
# Training 
model_cv = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)
model_cv.build_vocab(tagged_data_cv)
model_cv.train(tagged_data_cv, total_examples=model_cv.corpus_count, epochs=model_cv.epochs)

model_vacancies = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)
model_vacancies.build_vocab(tagged_data_vacancies)
model_vacancies.train(tagged_data_vacancies, total_examples=model_vacancies.corpus_count, epochs=model_vacancies.epochs)

In [80]:
similarities = []
for i in range(len(CVs)):
    similarity_scores = model_vacancies.docvecs.most_similar([model_cv.infer_vector(word_tokenize(CVs['Processed_Resume'][i]))], topn=1)
    similarities.append((i, similarity_scores[0][0], similarity_scores[0][1]))

C:\Users\Tseh\AppData\Local\Temp\ipykernel_18460\3399075100.py:3: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similarity_scores = model_vacancies.docvecs.most_similar([model_cv.infer_vector(word_tokenize(CVs['Processed_Resume'][i]))], topn=1)


In [81]:
 similarities_df = pd.DataFrame(similarities, columns=['CV_Index', 'Matched_Vacancy_Index', 'Similarity_Score'])

matched_data = pd.merge(CVs, similarities_df, left_index=True, right_on='CV_Index')
matched_data = matched_data.merge(vacancies, left_on='Matched_Vacancy_Index', right_index=True)

In [82]:
 matched_data_final = matched_data[['Processed_Resume', 'Processed_Description']]

                                        Processed_Resume  \
0      системный администратор советск калининградска...   
1497   менеджер сопровождению клиентов готов переезду...   
1      технический писатель королев готов переезду го...   
5366   руководитель группы технической поддержки моск...   
17347  ведущий программист сочи готова переезду москв...   
...                                                  ...   
19983  менеджер ярославль готов переезду готов редким...   
19985  махачкала готов переезду готов командировкам ч...   
19993  оператор группы ввода данных маркета московска...   
19995  технический специалист екатеринбург чкаловская...   
19999  монтажник слаботочных систем москва бунинская ...   

                                   Processed_Description  
0      менеджер контроля качества отдел аналитики опы...  
1497   менеджер контроля качества отдел аналитики опы...  
1      frontend engineer злым марсианам удаленная раб...  
5366   frontend engineer злым марсианам уда

In [84]:
matched_data_final.head(50)

,Processed_Resume,Processed_Description
0,системный администратор советск калининградска...,менеджер контроля качества отдел аналитики опы...
1497,менеджер сопровождению клиентов готов переезду...,менеджер контроля качества отдел аналитики опы...
1,технический писатель королев готов переезду го...,frontend engineer злым марсианам удаленная раб...
5366,руководитель группы технической поддержки моск...,frontend engineer злым марсианам удаленная раб...
17347,ведущий программист сочи готова переезду москв...,frontend engineer злым марсианам удаленная раб...
2,оператор тверь готова переезду готова командир...,системный аналитик нефтеперерабатывающий завод...
15806,ведущий специалист технической поддержке липец...,системный аналитик нефтеперерабатывающий завод...
3,html cs j php базы данных фреймворки дизайн ин...,специалист службы поддержки техническими знани...
345,волгоград готов переезду готов командировкам п...,специалист службы поддержки техническими знани...
647,москва теплый стан готов переезду готов команд...,специалист службы поддержки техническими знани...


# Word2Vec

In [90]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

In [88]:
combined_corpus = list(CVs['Processed_Resume']) + list(vacancies['Processed_Description'])
word2vec_model = Word2Vec(sentences=combined_corpus, vector_size=100, window=5, min_count=1, workers=4)

In [89]:
def get_document_vector(tokens, model):
    vector = []
    for word in tokens:
        if word in model.wv:
            vector.append(model.wv[word])
    if vector:
        return sum(vector) / len(vector)
    else:
        return [0] * model.vector_size

In [91]:
CVs['Vector_Resume'] = CVs['Processed_Resume'].apply(lambda x: get_document_vector(x, word2vec_model))
vacancies['Vector_Description'] = vacancies['Processed_Description'].apply(lambda x: get_document_vector(x, word2vec_model))

In [92]:
similarity_matrix = cosine_similarity(list(CVs['Vector_Resume']), list(vacancies['Vector_Description']))

In [93]:
matched_indices = similarity_matrix.argmax(axis=1)
CVs['Matched_Vacancy_ID'] = matched_indices
CVs['Matched_Vacancy_Description'] = vacancies.loc[matched_indices, 'vacancies'].values

In [94]:
matched_data = pd.merge(CVs, vacancies, left_on='Matched_Vacancy_ID', right_index=True, suffixes=('_Resume', '_Vacancy'))
matched_data_final = matched_data[['Processed_Resume', 'Processed_Description']]

In [96]:
matched_data_final.head(50)

,Processed_Resume,Processed_Description
0,системный администратор советск калининградска...,райтер журналист года лет полный день новостей...
1855,системный аналитик тестировщик qa специалист о...,райтер журналист года лет полный день новостей...
5225,специалист контролю качества волгоград готов п...,райтер журналист года лет полный день новостей...
5945,системный аналитик тестировщик qa специалист о...,райтер журналист года лет полный день новостей...
6874,специалист it системный администратор новосиби...,райтер журналист года лет полный день новостей...
7016,специалист it алматы готов переезду готов редк...,райтер журналист года лет полный день новостей...
7071,руководитель отдела маркетинга маркетолог seo ...,райтер журналист года лет полный день новостей...
8998,специалист технической поддержки руководитель ...,райтер журналист года лет полный день новостей...
11338,консультант сопровождению красноярск готов пер...,райтер журналист года лет полный день новостей...
11814,оператор бухгалтер волжский готова переезду го...,райтер журналист года лет полный день новостей...
